In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import os
import csv
import numpy as np
import gc
root_dir = os.path.abspath(os.curdir)
pred_dir_subs = os.path.dirname(root_dir)+"/SubmissionPredictions/"
pred_dir_coms = os.path.dirname(root_dir)+"/CommentPredictions/"
#Create directory to save experiment results
results_dir = os.path.dirname(root_dir)+"/Results/"
os.makedirs(results_dir, exist_ok=True)

In [2]:
def getToxicityChanges(collection,pred_dir):
    print(collection)
    for year in range(2006,2008):
        print(year)
        con1 = 0
        con2 = 0
        same = 0
        diff = 0
        counter = 0
        conditions = []
        chkcounter = 0

        df = pd.read_csv(pred_dir+str(year)+'.csv',usecols=['Author','Subreddit','Time','Label'])
        output = results_dir+collection+'-changesInToxicity-'+str(year)+'.csv'
        with open(output,'a') as out:
            out.write("Username,con1,con2,same3,diff4,numChanges\n")
            for author, data in df.groupby("Author"):
                if len(data) > 1:
                    data['Time'] = pd.to_datetime(data.Time)
                    data = data.sort_values(by='Time')
                    data.to_csv("temp.csv",index=False)
                    f = open("temp.csv", "r+",encoding='utf-8')
                    #remove header
                    next(f)
                    ff = csv.reader(f)
                    pre_line = next(ff)
                    while(True):
                        counter = counter + 1
                        try:
                            cur_line = next(ff)
                            if pre_line[3] == 'non_toxic' and cur_line[3] == 'slightly_toxic':

                                con1 = 1
                                conditions.append(con1)
                                if pre_line[1] == cur_line[1]:
                                    same = 3
                                    conditions.append(same)
                                else:
                                    diff = 4
                                    conditions.append(diff)

                            elif pre_line[3] == 'non_toxic' and cur_line[3] == 'highly_toxic':

                                con1 = 1
                                conditions.append(con1)
                                if pre_line[1] == cur_line[1]:
                                    same = 3
                                    conditions.append(same)
                                else:
                                    diff = 4
                                    conditions.append(diff)

                            elif pre_line[3] == 'highly_toxic' and cur_line[3] == 'non_toxic':

                                con2 = 2
                                conditions.append(con2)
                                if pre_line[1] == cur_line[1]:
                                    same = 3
                                    conditions.append(same)
                                else:
                                    diff = 4
                                    conditions.append(diff)

                            elif pre_line[3] == 'slightly_toxic' and cur_line[3] == 'non_toxic':

                                con2 = 2
                                conditions.append(con2)
                                if pre_line[1] == cur_line[1]:
                                    same = 3
                                    conditions.append(same)
                                else:
                                    diff = 4
                                    conditions.append(diff)

                            pre_line = cur_line
                        except:
                            break


                    changes = []
                    if conditions:
                        countsDF=pd.DataFrame({'Number': conditions})
                        df1 = pd.DataFrame(data=countsDF['Number'].value_counts(), columns=['Number','Count'])
                        df1['Count']=df1['Number'].index 
                        conds = df1.loc[df1.index.intersection([1,2])]
                        locations = df1.loc[df1.index.intersection([3,4])]

                        maxCon = list(conds[conds['Number']==conds.Number.max()]['Count'])
                        maxLoc = list(locations[locations['Number']==locations.Number.max()]['Count'])

                        changes.append(len(conditions)/2)
                        finalRes = maxCon + maxLoc + changes

                        out.write(author+","+",".join(str(item) for item in finalRes)+"\n")

                        del conditions[:]
                        del changes[:]
                    else:
                        out.write(author+",No change!\n")
                    con1 = 0
                    con2 = 0
                    same = 0
                    diff = 0
        del df
        gc.collect()

def combineChangesInToxicity(collection):
    output = results_dir+collection+'-changesInToxicity-combinedChanges.csv'
    with open(output,'a') as out:
        out.write("Username,con1,con2,same3,diff4,numChanges\n")
        for year in range(2006,2008):
            file = results_dir+collection+'-changesInToxicity-'+str(year)+'.csv'
            df = pd.read_csv(file)
            df = df[df['con1'] != 'No change!']
            df.to_csv(out,index=False,header=False,line_terminator='\n')
            
def groupCombinedChanges(collection):
    changesFile = results_dir+collection+'-changesInToxicity-combinedChanges.csv'
    df = pd.read_csv(changesFile)   
    output = results_dir+collection+'-changesInToxicity-combinedChanges-grouped.csv'
    with open(output,'a') as out:
        out.write("Username,con1,con2,same3,diff4,numChanges\n")
        for author, data in df.groupby("Username"):
            data = data.dropna(axis=1, how='all')
            data.iloc[:, -1].fillna(data.iloc[:, -2], inplace=True)
            totalChanges = sum(data.iloc[:, -1].fillna(0))
            dataList = data.iloc[:,1:-1].stack().tolist()
            dataList = list(set([int(x) for x in dataList]))
            dataList = [x for x in dataList if x <= 4]
            dataString = ','.join(str(item) for item in dataList)
            outputString = author+','+dataString+','+str(totalChanges)+'\n'
            out.write(outputString)

def countChangesInToxicity(collection):
    file = results_dir+collection+'-changesInToxicity-combinedChanges-grouped.csv'
    df = pd.read_csv(file)

    del df['numChanges']
    df = df[df['con1'] != 'No change!']
    df = df.fillna(0)
    df['con1'] = df['con1'].astype(np.int64)
    df['con2'] = df['con2'].astype(np.int64)
    df['same3'] = df['same3'].astype(np.int64)
    df['diff4'] = df['diff4'].astype(np.int64)
    print(df)
  
    con1 = df[~df.eq(2).any(1)]
    print("condition 1=",len(con1),format((len(con1)/len(df))*100,'.2f'))
    con2 = df[~df.eq(1).any(1)]
    print("condition 2=",len(con2),format((len(con2)/len(df))*100,'.2f'))
    #same sub con1
    same1 = con1[~con1.eq(4).any(1)]
    print("condition 1 same =",len(same1),format((len(same1)/len(con1))*100,'.2f'))
    #same sub con2
    if len(con2) != 0:
        same2 = con2[~con2.eq(4).any(1)]
        print("condition 2 same =",len(same2),format((len(same2)/len(con2))*100,'.2f'))
    #diff sub con1
    diff1 = con1[~con1.eq(3).any(1)]
    print("condition 1 diff =",len(diff1),format((len(diff1)/len(con1))*100,'.2f'))
    #diff sub con2
    if len(con2) != 0:
        diff2 = con2[~con2.eq(3).any(1)]
        print("condition 2 diff =",len(diff2),format((len(diff2)/len(con2))*100,'.2f'))
        
    bothConditions =  df [df.eq(1).any(1) & df.eq(2).any(1)]
    print("both conditions=",len(bothConditions),format((len(bothConditions)/len(df))*100,'.2f'))
    bothSame = bothConditions[~bothConditions.eq(4).any(1)]
    print("same subreddit=",len(bothSame),format((len(bothSame)/len(bothConditions))*100,'.2f'))
    bothDiff = bothConditions[~bothConditions.eq(3).any(1)]
    theLen = len(bothConditions)- len(bothSame)
    print("different subreddits=",theLen,format((theLen/len(bothConditions))*100,'.2f'))
    sameDiffBoth = len(bothConditions)-(len(bothSame)+len(bothDiff))
    print("Both conditions in same and different subs=",sameDiffBoth,format((sameDiffBoth/theLen)*100,'.2f'))

In [3]:
getToxicityChanges("Submission",pred_dir_subs)
if os.path.exists("temp.csv"):
    os.remove("temp.csv")
combineChangesInToxicity("Submission")
groupCombinedChanges("Submission")
countChangesInToxicity("Submission")
getToxicityChanges("Comment",pred_dir_coms)
if os.path.exists("temp.csv"):
    os.remove("temp.csv")
combineChangesInToxicity("Comment")
groupCombinedChanges("Comment")
countChangesInToxicity("Comment")

Submission
2006
2007
         Username  con1  con2  same3  diff4
0          9jack9     1     2      3      4
1         AGivant     1     2      3     10
2    AbouBenAdhem     1     2      3      4
3           Aerik     1     2      3    522
4         AlDente     1     2      3      6
..            ...   ...   ...    ...    ...
353        yurbud     1     2      3      4
354       zangorn     1     2      3      2
355           zem     1     2      3      4
356      zer0mike     1     2      3      2
357        zoinks     2     4      1      0

[358 rows x 5 columns]
condition 1= 45 12.57
condition 2= 13 3.63
condition 1 same = 39 86.67
condition 2 same = 12 92.31
condition 1 diff = 5 11.11
condition 2 diff = 0 0.00
both conditions= 300 83.80
same subreddit= 211 70.33
different subreddits= 89 29.67
Both conditions in same and different subs= 70 78.65
Comment
2006
2007
        Username  con1  con2  same3  diff4
0            -J-     1     3     11      0
1            02J     1     2      